<a href="https://colab.research.google.com/github/Xiyao-Song/Xiyao-Song/blob/main/PgAdmin_DB_Analysis_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Connection cell

In [ ]:
# step 0: setting up the connection
!pip install -U "psycopg[binary]"

import psycopg
import os

conn = psycopg.connect(
    host="localhost",   # Using what we did in class and hw2
    port="5432",
    dbname="Yelp_Project",   # Guys you should create this DB in pgAdmin first
    user="postgres",
    password="123"               # <-- I used 123, dont forget to change to your own
)

cur = conn.cursor()

print("PostgreSQL database version:")
cur.execute("SELECT version()")
print(cur.fetchone())

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 15.2 MB/s  0:00:00

  Attempting uninstall: psycopg-binary

    Found existing installation: psycopg-binary 3.2.10

    Uninstalling psycopg-binary-3.2.10:

      Successfully uninstalled psycopg-binary-3.2.10

   ---------------------------------------- 0/2 [psycopg-binary]
  Attempting uninstall: psycopg
   ---------------------------------------- 0/2 [psycopg-binary]
    Found existing installation: psycopg 3.2.10
   ---------------------------------------- 0/2 [psycopg-binary]
    Uninstalling psycopg-3.2.10:
   ---------------------------------------- 0/2 [psycopg-binary]
      Successfully uninstalled psycopg-3.2.10
   ---------------------------------------- 0/2 [psycopg-binary]
   -------------------- ------------------- 1/2 [psycopg]
   -------------------- ------------------- 1/2 [psycopg]
   -------------------- ------------------- 1/2 [psycopg]
   ----

### Create tables in Postgres

#### 1. Yelp amenities table
only keep columns: id, name, location, rating

In [ ]:
# step 1: create tables

create_yelp_table = """
CREATE TABLE IF NOT EXISTS yelp_amenities (
    business_id       TEXT PRIMARY KEY,
    name              TEXT,
    city              TEXT,
    state             TEXT,
    latitude          DOUBLE PRECISION,
    longitude         DOUBLE PRECISION,
    stars             DOUBLE PRECISION,
    review_count      INTEGER,
    primary_category  TEXT,
    popularity_score  DOUBLE PRECISION,
    amenity           TEXT
);
"""
cur.execute(create_yelp_table)

create_landuse_stats = """
CREATE TABLE IF NOT EXISTS landuse_restaurant_stats (
    landuse_simple  TEXT PRIMARY KEY,
    n_restaurants   INTEGER,
    avg_stars       DOUBLE PRECISION,
    avg_reviews     DOUBLE PRECISION
);
"""
cur.execute(create_landuse_stats)

conn.commit()

#### Load CSVs

In [ ]:
import pandas as pd

# adjust the path to wherever the file is on your laptop/jupyter home
amen_path = "merged_yelp_ammenities.csv"

amen_df = pd.read_csv(
    amen_path,
    usecols=[
        "business_id", "name_left", "city", "state",
        "latitude", "longitude", "stars", "review_count",
        "primary_category", "popularity_score", "amenity"
    ]
)

# rename to match the table column "name"
amen_df = amen_df.rename(columns={"name_left": "name"})

# convert to list of tuples
rows = list(amen_df.itertuples(index=False, name=None))
len(rows)

34605

In [ ]:
insert_yelp = """
INSERT INTO yelp_amenities (
    business_id, name, city, state,
    latitude, longitude,
    stars, review_count,
    primary_category, popularity_score,
    amenity
) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
ON CONFLICT (business_id) DO NOTHING;
"""

cur.executemany(insert_yelp, rows)
conn.commit()
print("Inserted rows into yelp_amenities.")

Inserted rows into yelp_amenities.


#### Load philly_landuse_restaurant_stats.csv → landuse_restaurant_stats

In [ ]:
stats_path = "philly_landuse_restaurant_stats.csv"
stats_df = pd.read_csv(stats_path)

rows_stats = list(stats_df.itertuples(index=False, name=None))

insert_stats = """
INSERT INTO landuse_restaurant_stats (
    landuse_simple, n_restaurants, avg_stars, avg_reviews
) VALUES (%s,%s,%s,%s)
ON CONFLICT (landuse_simple) DO UPDATE
SET n_restaurants = EXCLUDED.n_restaurants,
    avg_stars      = EXCLUDED.avg_stars,
    avg_reviews    = EXCLUDED.avg_reviews;
"""

cur.executemany(insert_stats, rows_stats)
conn.commit()
print("Inserted rows into landuse_restaurant_stats.")

Inserted rows into landuse_restaurant_stats.


### use SQL in the project

#### Top 10 categories in Philly by # of restaurants

In [ ]:
sql_top_categories = """
SELECT
    primary_category,
    COUNT(*) AS num_restaurants,
    ROUND(AVG(stars)::numeric, 2) AS avg_rating
FROM yelp_amenities
WHERE city = 'Philadelphia'
GROUP BY primary_category
HAVING COUNT(*) >= 20
ORDER BY num_restaurants DESC
LIMIT 10;
"""

top_categories = pd.read_sql(sql_top_categories, conn)
top_categories

C:\Users\songx\AppData\Local\Temp\ipykernel_4468\2202888178.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_categories = pd.read_sql(sql_top_categories, conn)


,primary_category,num_restaurants,avg_rating
0,Restaurants,1731,3.51
1,Food,764,3.66
2,Bars,215,3.51
3,Coffee & Tea,197,3.55
4,Fast Food,85,2.46
5,Specialty Food,79,4.08
6,Seafood,66,3.43
7,Desserts,65,3.91
8,Barbers,55,4.05
9,Sushi Bars,52,3.73


#### Rating vs popularity (checkins) in Philly

In [ ]:
sql_popularity = """
SELECT
    popularity_score,
    stars
FROM yelp_amenities
WHERE city = 'Philadelphia'
  AND popularity_score IS NOT NULL;
"""

popularity_df = pd.read_sql(sql_popularity, conn)
popularity_df.head()

C:\Users\songx\AppData\Local\Temp\ipykernel_4468\500554751.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  popularity_df = pd.read_sql(sql_popularity, conn)


,popularity_score,stars
0,377.50,4.0
1,298.75,4.0
2,116.25,4.0
3,201.25,2.5
4,422.50,3.0


#### Land use summary table (already aggregated)

In [ ]:
sql_landuse = """
SELECT
    landuse_simple,
    n_restaurants,
    ROUND(avg_stars::numeric, 2)  AS avg_stars,
    ROUND(avg_reviews::numeric, 1) AS avg_reviews
FROM landuse_restaurant_stats
ORDER BY avg_stars DESC;
"""

landuse_summary = pd.read_sql(sql_landuse, conn)
landuse_summary

C:\Users\songx\AppData\Local\Temp\ipykernel_4468\3036081856.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  landuse_summary = pd.read_sql(sql_landuse, conn)


,landuse_simple,n_restaurants,avg_stars,avg_reviews
0,residential,101,3.84,93.2
1,unknown,3159,3.60,90.1
2,other,19,3.58,69.7
3,park/recreation,9,3.39,19.4
4,commercial/retail,724,3.35,74.1


We exported our processed Yelp and land-use data to CSV and loaded them into a PostgreSQL database using the psycopg Python library.
We created normalized tables yelp_amenities (individual Yelp businesses with ratings, categories, and popularity scores) and landuse_restaurant_stats (aggregated restaurant statistics by land-use type).
Using SQL SELECT queries with GROUP BY, ORDER BY, and filtering, we computed metrics such as:

Top Yelp categories in Philadelphia by restaurant count and average rating.

Average Yelp ratings by simplified land-use category.
The query results were read back into pandas and used to generate the tables and charts in our final analysis.